# 重回帰分析
重回帰モデルを用いて，消費電力と気象データ（最高気温，最低気温、湿度）との関係を分析する。<br>
電力使用量は東京電力管区内，気温の測定地点は東京，地域性の設定に問題があるが，練習問題としてご了解願いたい。

## データの取得方法
### 消費電力
#### 要件
2022年1月1日～2022年12月31日までの電力使用データを用いる。これは1時間ごとのデータである。気象データに合わせるため，1日ごとのデータにダウンサンプリング（down-sampling）する。この仕方は，その日の最大消費電力[万kW]を採用する。
#### 手順
1. 東京電力パワーグリッド（株）の「でんき予報」サイトを開く。
    - site:https://www.tepco.co.jp/  内で"でんき予報"を検索
2. [過去の電力使用実績データ]のWebページに移り[2022年]をクリックして，CSVファイル"juyo-2022.csv"をダウンロード
3. この内容で，不要な部分は'#"を付けてコメント行とし，"EPower2022.csv"(encoding='SHIFT-JIS')として保存。
4. ダウンサンプリングは，下記のスクリプト中で行う。

### 気象データ（最高気温，最低気温，相対湿度）
#### 要件
日ごとの3データ（最高気温，最低気温，相対湿度）を得る。相対湿度とは，一般で言う湿度のことである。https://ja.wikipedia.org/wiki/相対湿度
#### 手順
1. 気象庁のサイトの中の「過去の気象データ・ダウンロード」サイトを開く
    - site:https://www.data.jma.go.jp/ 内で"過去の気象データ・ダウンロード"を検索
2. [地点を選ぶ]から[東京]->[東京]を選択
3. [項目を選ぶ]から[データの種類] &rarr; [日別値]，[項目] &rarr; [気温]から[日最高気温]と[日最低気温]，[湿度／気温]から[日平均相対湿度]を選ぶ。
4. [期間を選ぶ] &rarr; 2022年1月1日～2022年12月31日に設定する。
5. [CSVファイルをダウンロード]をクリックして，ファイルを得る。
6. このファイルを手作業で加工して，"dataWeatherTokyo.csv"（encoding=UTF-8）として保存。




In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_pow = pd.read_csv('dataEPower2022.csv', comment='#', 
                    index_col='DATE', parse_dates=['DATE'],  encoding='SHIFT-JIS' )
df_pow.head(3)

In [ ]:
df_pow.tail(3)

#### ダウンサンプリング
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html  
時間軸を日単位として，最大電力だけとする

In [ ]:
df_pow_down = df_pow.resample('D').max() # Dayへのダウンサンプリングと最大電力
print(df_pow_down.head(3))
print(df_pow_down.shape)

In [ ]:
df_pow_down.Power['2022-04-30']

#### 最高気温，最低気温のデータ読込み

In [ ]:
df = pd.read_csv('dataWeatherTokyo.csv', comment='#', index_col='Date', parse_dates=['Date'])
print(df.head())
print(df.shape)

#### データの結合

In [ ]:
df['MaxPower'] = df_pow_down.Power
print(df.head(3))

#### 重回帰モデルの選定
次の4種を試し，R-squaredの値に注目する。
1. MaxPower ~ MaxTemp + MinTemp + Humid -1
2. MaxPower ~ MaxTemp + MinTemp + Humid
3. MaxPower ~ MaxTemp + MinTemp -1
4. MaxPower ~ MaxTemp + MinTemp

上記の3と4は，単に試しただけで，深い意味は無い。

In [ ]:
result1 = smf.ols('MaxPower ~ MaxTemp + MinTemp + Humid -1', data=df).fit()
print(result1.summary())

In [ ]:
result2 = smf.ols('MaxPower ~ MaxTemp + MinTemp + Humid', data=df).fit()
print(result2.summary())

In [ ]:
result3 = smf.ols('MaxPower ~ MaxTemp + MinTemp -1', data=df).fit()
print(result3.summary())

In [ ]:
result4 = smf.ols('MaxPower ~ MaxTemp + MinTemp', data=df).fit()
print(result4.summary())

In [ ]:
pred2 = result1.predict(df)

In [ ]:
pred2.head()

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6,9))
df.MaxPower.plot(ax=ax[0])
pred2.plot(ax=ax[0], label='pred2')
#df.plot(ax=ax[0])
df.MaxTemp.plot(ax=ax[1])
df.MinTemp.plot(ax=ax[1])
df.Humid.plot(ax=ax[1])
ax[0].grid()
ax[1].grid()
ax[0].legend()
ax[1].legend()

#plt.savefig('fig_REG_Power_Weather_ex01.pdf', bbox_inches='tight')
plt.show()

#### 二つの期間を抽出して再計算

In [ ]:
df1st = df['2022/1/15':'2022/4/30'] # 1st
df2nd = df['2022/5/1':'2022/8/31']  # 2nd
print(df1st.shape, df2nd.shape)

In [ ]:
result_1st = smf.ols('MaxPower ~ MaxTemp + MinTemp + Humid -1', data=df1st).fit()
print(result_1st.summary())

In [ ]:
pred_1st = result_1st.predict(df1st)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6,8))
df1st.MaxPower.plot(ax=ax[0])
pred_1st.plot(ax=ax[0], label='pred_1st')
#df.plot(ax=ax[0])
df1st.MaxTemp.plot(ax=ax[1])
df1st.MinTemp.plot(ax=ax[1])
df1st.Humid.plot(ax=ax[1])
ax[0].grid()
ax[1].grid()
ax[0].legend()
ax[1].legend()

In [ ]:
result_2nd = smf.ols('MaxPower ~ MaxTemp + MinTemp + Humid -1', data=df2nd).fit()
print(result_2nd.summary())
pred_2nd = result_2nd.predict(df2nd)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6,8))
df2nd.MaxPower.plot(ax=ax[0])
pred_2nd.plot(ax=ax[0], label='pred_2nd')
#df.plot(ax=ax[0])
df2nd.MaxTemp.plot(ax=ax[1])
df2nd.MinTemp.plot(ax=ax[1])
df2nd.Humid.plot(ax=ax[1])
ax[0].grid()
ax[1].grid()
ax[0].legend()
ax[1].legend()

In [ ]:
fig = plt.subplots(figsize=(6,3))
df2nd.MaxPower.plot()
pred_2nd.plot(label='pred_2nd')
plt.grid()
plt.legend()
#plt.savefig('fig_REG_Power_Weather_ex02.pdf', bbox_inches='tight')
plt.show()